# NLP RECSYS Run Pipeline Stage1




In [1]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import org.apache.spark.ml.{Pipeline, PipelineModel, Transformer}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.storage._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.ml.param.{Param, ParamMap}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, explode, udf}
import org.apache.spark.sql.types.{DataTypes, StructType}

val dataDir = sys.env("HOME") + "/recsys2020"

In [4]:
package nlprecsys
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import org.apache.spark.ml.{Pipeline, PipelineModel, Transformer}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.storage._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.ml.param.{Param, ParamMap}
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, explode, udf}
import org.apache.spark.sql.types.{DataTypes, StructType}

class Exploder(override val uid: String) extends Transformer with DefaultParamsWritable {
  def this() = this(Identifiable.randomUID("Exploder"))
  def setInputCol(value: String): this.type = set(inputCol, value)
  def setOutputCol(value: String): this.type = set(outputCol, value)
  def getOutputCol: String = getOrDefault(outputCol)
  val inputCol = new Param[String](this, "inputCol", "input column")
  val outputCol = new Param[String](this, "outputCol", "output column")

  override def transform(dataset: Dataset[_]): DataFrame = {
    val outCol = extractParamMap.getOrElse(outputCol, "output")
    val inCol = extractParamMap.getOrElse(inputCol, "input")
    dataset.withColumn(outCol, explode(col(inCol)))
  }

  override def transformSchema(schema: StructType): StructType = {
      val outCol = extractParamMap.getOrElse(outputCol, "output")
      val inCol = extractParamMap.getOrElse(inputCol, "input")
      val inputColType = schema.fields(schema.fieldIndex(inCol)).dataType.asInstanceOf[ArrayType];
      schema.add(outCol, inputColType.elementType)
  }
  override def copy(extra: ParamMap): Transformer = defaultCopy(extra)
}
object Exploder extends DefaultParamsReadable[Exploder] {
  override def load(path: String): Exploder = super.load(path)
}



In [3]:
val pipeline = PipelineModel.load(dataDir + "/pipeline_stage1_v1")

In [2]:
val dsName = "training"

val classNames = Array(
  "retweet",
  "retweet_with_comment",
  "like",
  "reply")
val labelColumns = for (className <- classNames) yield "has_" + className;
val df = spark.read.parquet(dataDir + s"/${dsName}.parquet")
val trans_df = pipeline.transform(df).selectExpr((Array("tweet_id", "user_id", "features") ++ labelColumns):_*)

trans_df
    .write
    .mode(SaveMode.Overwrite)
    .parquet(dataDir + s"/${dsName}_stage1.parquet")

In [5]:
trans_df.show()

+--------------------+--------------------+--------------------+-----------+------------------------+--------+---------+
|            tweet_id|             user_id|            features|has_retweet|has_retweet_with_comment|has_like|has_reply|
+--------------------+--------------------+--------------------+-----------+------------------------+--------+---------+
|FB6304C97F6CC05AF...|D0EA9DDFE93EDA782...|[0.06471998244524...|          0|                       0|       1|        0|
|8A025814338D4CAB5...|5CD17BD84873464C2...|[0.01755615323781...|          0|                       0|       0|        0|
|8E8EC11B9492B6C09...|6B8C5EFC300170EF8...|[0.02887635119259...|          0|                       0|       0|        0|
|D59F5C535C7C892D7...|2F5D9BFEBEB24F290...|[0.05812061950564...|          0|                       0|       0|        0|
|7D6700DA02116E216...|073770D7360B4F399...|[0.07400916516780...|          0|                       0|       1|        0|
|E5A3EC400A664F07A...|1318151966